<a href="https://colab.research.google.com/github/Yanis52/Skin-Cancer-Detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#test
#!pip install kagglehub
import kagglehub

In [6]:
# path = kagglehub.dataset_download("nodoubttome/skin-cancer9-classesisic")

print("Path to dataset files:", path)

100%|██████████| 786M/786M [00:09<00:00, 85.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/nodoubttome/skin-cancer9-classesisic/versions/1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


dataset available on : /root/.cache/kagglehub/datasets/nodoubttome/skin-cancer9-classesisic/versions/1